In [ ]:
pip install -r requirements.txt

In [1]:
import os

os.environ["TOGETHER_API_KEY"] = 'daacc8dc45f272f48e8571c2ff9bbccc7169541e632faa75e7efa12900cf2813'


In [13]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings.together import TogetherEmbedding
from llama_index.llms.together import TogetherLLM
import warnings
warnings.filterwarnings('ignore')


# Provide a template following the LLM's original chat template.
def completion_to_prompt(completion: str) -> str:
  return f"<s>[INST] {completion} [/INST] </s>\n"


def run_rag_completion(
    document_dir: str,
    query_text: str,
    embedding_model: str ="togethercomputer/m2-bert-80M-8k-retrieval",
    generative_model: str ="mistralai/Mixtral-8x7B-Instruct-v0.1"
    ) -> str:
    service_context = ServiceContext.from_defaults(
        llm=TogetherLLM(
            generative_model,
            temperature=0.0,
            max_tokens=256,
            top_p=0.9,
            top_k=35,
            repetition_penalty=1.1,
            is_chat_model=True,
            completion_to_prompt=completion_to_prompt
        ),
        embed_model=TogetherEmbedding(embedding_model)
    )
    documents = SimpleDirectoryReader(document_dir).load_data()
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    response = index.as_query_engine(similarity_top_k=5).query(query_text)

    return str(response)


query_text = "What is this document about?"
document_dir = './Documents'

response = run_rag_completion(document_dir, query_text)
print(response)



 This document is a project proposal for an inspection assistance system, specifically designed for the manufacturing sector in the automotive industry, with a focus on inspecting Electric Vehicle (EV) batteries. The core concept involves utilizing a wearable device, such as smart glasses, to capture live video feeds of the battery from multiple angles. The system will then overlay any detected defects or anomalies onto the user's view in real-time, enhancing the efficiency of the inspection process. The document delves into various components of the project, including the system's architecture, software requirements, features, functional specifications, and implementation strategies.
